# Phase III: First ML Proof of Concept (5\%)

### Team Names:
- Luc
- Nora
- Kevin
- Thomas
- Zijiang Yang

In [1]:
# Importing all useful libraries (for webscraping and ML as well)
import os
import pandas as pd
import numpy as np
import requests
import time
from io import BytesIO

In [ ]:
#NOTE: This takes some time to run (it took me 6 minutes to finish, but it may be faster or slower)

# webscraping
api_key = "03bcc17f7d105b13199e0325b659d4ab"
API_KEY = "6c30ec8965840313bb630941ccabc149"
BASE_URL = "https://api.themoviedb.org/3"

# Function to get movie details by ID
def get_movie_details(movie_id):
    """
    Gets the movie details of a movie through a movie id.

    Args:
        movie_id (string): the id of the movie as a string.
    
    Returns:
        response: the reponse (as a json) recieved.
    """
    url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching details for movie ID {movie_id}: {response.status_code}")
        return None

# Function to get movie credits by ID
def get_movie_credits(movie_id):
    """
    Gets the movie credits of a movie through a movie id.

    Args:
        movie_id (string): the id of the movie as a string.
    
    Returns:
        response: the reponse (as a json) recieved.
    """
    url = f"{BASE_URL}/movie/{movie_id}/credits?api_key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching credits for movie ID {movie_id}: {response.status_code}")
        return None

# Function to get actor popularity by ID
def get_actor_popularity(actor_id):
    """
    Gets the popularity of an actor through an actor id.

    Args:
        actor_id (string): the id of the actor as a string.
    
    Returns:
        response: the popularity of the actor.
    """
    url = f"{BASE_URL}/person/{actor_id}?api_key={API_KEY}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        return data.get('popularity')
    else:
      print(f"Error fetching popularity for actor ID {actor_id}: {response.status_code}")
      return None

page = 1
df = pd.DataFrame()
for i in range(50): #the number of pages to go up until
    url = f"{BASE_URL}/movie/top_rated?api_key={api_key}&language=en-US&page={page}"

    response = requests.get(url)
    data = response.json()

    new_data = pd.DataFrame(data['results'])

    df = pd.concat([df, new_data])
    page += 1

# Filter out movies with vote count less than 1000
df = df[df['vote_count'] >= 1000].copy() # Use .copy() to avoid SettingWithCopyWarning

# Add budget information
df['budget'] = None
for index, row in df.iterrows():
    movie_id = row['id']
    details = get_movie_details(movie_id)
    if details and 'budget' in details:
        df.loc[index, 'budget'] = details['budget']
    time.sleep(0.05) # Add a small delay to avoid hitting API limits

# Add top actor popularity
df['top_actor_popularity (out of 10)'] = None # Initialize top_actor_popularity column
for index, row in df.iterrows():
    movie_id = row['id']
    credits = get_movie_credits(movie_id)
    if credits and 'cast' in credits and len(credits['cast']) > 0:
        top_actor_id = credits['cast'][0]['id']
        actor_popularity = get_actor_popularity(top_actor_id)
        df.loc[index, 'top_actor_popularity (out of 10)'] = actor_popularity
    time.sleep(0.05) # Add a small delay to avoid hitting API limits


# Convertin budget to millions n dropping orginal budget column
df['budget_in_millions'] = df['budget'] / 1000000
df = df.drop(columns=['budget'])

df.head()

,adult,backdrop_path,genre_ids,id,original_language,original_title,overview,popularity,poster_path,release_date,title,video,vote_average,vote_count,top_actor_popularity (out of 10),budget_in_millions
0,False,/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg,"[18, 80]",278,en,The Shawshank Redemption,Imprisoned in the 1940s for the double murder ...,25.0715,/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg,1994-09-23,The Shawshank Redemption,False,8.711,29225,0.452,7.275
1,False,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,"[18, 80]",238,en,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...",21.6694,/3bhkrj58Vtu7enYsRolD1fZdja1.jpg,1972-03-14,The Godfather,False,8.684,22070,2.2514,2.5
2,False,/kGzFbGhp99zva6oZODW5atUtnqi.jpg,"[18, 80]",240,en,The Godfather Part II,In the continuing saga of the Corleone crime f...,12.2619,/ecBRkXerAZqRRUfR8Lt3L3Dh6J5.jpg,1974-12-20,The Godfather Part II,False,8.571,13340,3.3393,0.0
3,False,/zb6fM1CX41D9rF9hdgclu0peUmy.jpg,"[18, 36, 10752]",424,en,Schindler's List,The true story of how businessman Oskar Schind...,10.4613,/sF1U4EUQS8YHUYjNl3pMGNIQyr0.jpg,1993-12-15,Schindler's List,False,8.566,16863,0.5835,15.0
4,False,/w4bTBXcqXc2TUyS5Fc4h67uWbPn.jpg,[18],389,en,12 Angry Men,The defense and the prosecution have rested an...,8.8201,/ow3wq89wM8qd5X7hWKxiRfsFf9C.jpg,1957-04-10,12 Angry Men,False,8.549,9531,0.3382,0.0


In [3]:
#clean data
def get_languages():
    """
    This function gets the list of movie languages from the TMDB.
    
    Returns:
        response (dict): The list of languages as a dictionary.
    """
    url = "https://api.themoviedb.org/3/configuration/languages"
    params = {"api_key": api_key}
    headers = {"accept": "application/json"}

    response = requests.get(url, headers=headers, params=params)
    return response.json()

def get_movie_genres():
    """
    This funciton gets the list of movie genres from the TMDB.
    
    Returns:
        response (dict): The list of genres as a dictionary.
    """
    url = "https://api.themoviedb.org/3/genre/movie/list?language=en"
    params = {"api_key": api_key}
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers, params=params)

    return response.json()

def clean_data(df):
    """
    Cleans the data given into these metrics: 
        - id
        - title
        - popularity
        - rating
        - number_of_raters
        - top_actor_popularity (out of 10)
        - budget_in_millions
        - release_year
        - release_month
        - release_day 
        - original language (as dummy variables in int (1) format)
        - genres (as dummy variables in int (1) format)

    Args:
        df (DataFrame): the inputted dataframe to be cleaned.

    Returns:
        cleaned_df (DataFrame): a cleaned dataframe.
    """
    
    #get the list of languages from TMDB
    languages_dict = get_languages()
    languages = {}
    for lang in languages_dict:
        languages[lang['iso_639_1']] = lang['english_name']
    
    #get the list of genres from TMDB
    genres_dict = get_movie_genres()
    genre_list = {}
    for genre in genres_dict['genres']:
        genre_list[genre['id']] = genre['name']

    #initialize lists to hold data
    movie_ids = []
    movie_titles = []
    movie_year = []
    movie_month = []
    movie_day = []
    movie_language = []
    movie_popularity = []
    movie_genres = []
    movie_vote_average = []
    movie_vote_count = []
    movie_budget = []
    movie_top_actor_popularity = []

    #loop through all rows of the data
    for data in df.iterrows():
        movie = data[1]

        movie_ids.append(movie['id']) #append movie id

        movie_titles.append(movie['title']) #append movie title

        #append movie release date
        movie_year.append(int(movie['release_date'][0:4])) 
        movie_month.append(int(movie['release_date'][5:7]))
        movie_day.append(int(movie['release_date'][8:10]))

        movie_language.append(languages[movie['original_language']]) #append movie language

        movie_popularity.append(float(movie['popularity'])) #append movie popularity

        #append movie genres as a list of genres
        genres = movie['genre_ids']
        list_of_genres = []
        for index in genres:
            list_of_genres.append(genre_list[index])
        movie_genres.append(list_of_genres)

        #append movie rating average and count
        movie_vote_average.append(float(movie['vote_average']))
        movie_vote_count.append(int(movie['vote_count']))

        movie_budget.append(round(float(movie['budget_in_millions']), 3)) #append movie budget in millions (rounded to 3 decimal places)

        movie_top_actor_popularity.append(movie['top_actor_popularity (out of 10)']) #append top actor popularity

    #put all of the lists into a dictionary
    df_dict = {'movie_id': movie_ids,
               'title': movie_titles,
               'popularity': movie_popularity,
               'rating': movie_vote_average,
               'number_of_raters': movie_vote_count,
               'top_actor_popularity (out of 10)': movie_top_actor_popularity,
               'budget_in_millions': movie_budget,
               'genres': movie_genres,
               'release_year': movie_year,
               'release_month': movie_month,
               'release_day': movie_day,
               'original_language': movie_language,
               }

    cleaned_df = pd.DataFrame(df_dict) #create the dataframe

    #make dummy variables for categorical data
    cleaned_df = pd.get_dummies(cleaned_df, columns=['original_language'], dtype='int')

    df_exploded = cleaned_df.explode('genres') #explode the genres column so each genre has its own row (we'll remove these extra rows later)
    df_genre_dummies = pd.get_dummies(df_exploded['genres']) #get the dummy variables for the genres
    cleaned_df = cleaned_df.join(df_genre_dummies.groupby(df_exploded.index).sum()) #put rows back together again

    cleaned_df = cleaned_df.drop(columns=['genres']) #drop the genres column now, as we don't need it

    cleaned_df = cleaned_df.sort_values(by='movie_id') #sort the dataframe by movie_id (this isn't really necessary)
    return cleaned_df.reset_index(drop=True) #return the dataframe, and reset the index of the rows

In [ ]:
# Print first 50 rows of cleaned data
cleaned_df = clean_data(df)
cleaned_df.head(50)

,movie_id,title,popularity,rating,number_of_raters,top_actor_popularity (out of 10),budget_in_millions,release_year,release_month,release_day,...,History,Horror,Music,Mystery,Romance,Science Fiction,TV Movie,Thriller,War,Western
0,11,Star Wars,14.0042,8.204,21652,3.4353,350.000,1977,5,25,...,0,0,0,0,0,1,0,0,0,0
1,12,Finding Nemo,12.7028,7.817,20051,3.0610,81.000,2003,5,30,...,0,0,0,0,0,0,0,0,0,0
2,13,Forrest Gump,17.4205,8.463,28860,4.1264,200.000,1994,6,23,...,0,0,0,0,1,0,0,0,0,0
3,14,American Beauty,6.1478,8.003,12643,4.2883,0.000,1999,9,15,...,0,0,0,0,0,0,0,0,0,0
4,15,Citizen Kane,4.3718,7.999,5812,0.3382,0.000,1941,4,17,...,0,0,0,1,0,0,0,0,0,0
5,16,Dancer in the Dark,3.8622,7.900,1908,0.4296,0.531,2000,9,1,...,0,0,0,0,0,0,0,0,0,0
6,19,Metropolis,2.9548,8.100,2964,2.3969,4.500,1927,2,6,...,0,0,0,0,0,1,0,0,0,0
7,22,Pirates of the Caribbean: The Curse of the Bla...,19.4297,7.815,21603,3.4353,350.000,2003,7,9,...,0,0,0,0,0,0,0,0,0,0
8,24,Kill Bill: Vol. 1,9.0556,7.970,18286,0.4296,0.531,2003,10,10,...,0,0,0,0,0,0,0,0,0,0
9,28,Apocalypse Now,7.0710,8.271,8734,0.4160,0.114,1979,5,19,...,0,0,0,0,0,0,0,0,1,0


## Part 1
(3%) The implementation (using NumPy) of your first ML model as a function call to the cleaned data

In [22]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy.stats import probplot, shapiro
import numpy as np
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [23]:
def add_bias_column(X):
    """
    Args:
        X (array): can be either 1-d or 2-d
    
    Returns:
        Xnew (array): the same array, but 2-d with a column of 1's in the first spot
    """
    
    # If the array is 1-d
    if len(X.shape) == 1:
        Xnew = np.column_stack([np.ones(X.shape[0]), X])
    
    # If the array is 2-d
    elif len(X.shape) == 2:
        bias_col = np.ones((X.shape[0], 1))
        Xnew = np.hstack([bias_col, X])
        
    else:
        raise ValueError("Input array must be either 1-d or 2-d")

    return Xnew

In [24]:
def line_of_best_fit(X, y):
    """
    Returns a vector containing the cooeficients of the line of best fit

    Args:
        X (array): array of X features
        y (array): array of target features
    Returns:
        m (vector): vector coefficients for the line of best fit
    """
    X = add_bias_column(X)
    XtXinv = np.linalg.inv(np.dot(X.T, X))
    m = np.dot(XtXinv, np.dot(X.T, y))
    return m

In [25]:
def linreg_predict(Xnew, ynew, m):
    """
    Predicts new values based on the line of best fit and returns data based on how well those values were predicted.
    
    Args:
        Xnew (array): can be either 1-d or 2-d
        ynew (array): 1-d array of target values
        m (vector): values for the line of best fit
    Returns:
        dict: {'ypreds':ypreds, 'resids':resids, 'mse':mse, 'r2':r2}
            ypreds (array): predicted values based on the line of best fit
            resids (array): residuals (ynew - ypreds)
            mse (float): mean squared error
            r2 (float): R-squared value
    """
    Xnew = add_bias_column(Xnew)

    ypreds = np.dot(Xnew, m)

    resids = ynew - ypreds

    mse = np.mean(resids ** 2)

    r2 = r2_score(ynew, ypreds)

    return {'ypreds':ypreds, 'resids':resids, 'mse':mse, 'r2':r2}

### Attempt 1 - Linear Regression

### Polynomial regression

Using the X features of top_actor_popularity, budget_in_millions, and release_month to create a model that determines the popularity of a movie. This model uses a polynomial with a degree of 2. 

### NOTE: I DONT THINK THIS MODEL IS VERY GOOD



In [270]:
list_of_features = []

#use either this method (excluding certain columns) or the method below (including certain columns) to get the X features

# columns_to_exclude = ['movie_id', 'title', 'rating', 'popularity', 'number_of_raters', 'release_year', 'release_day']
# for column in cleaned_df.columns:
#     if column not in columns_to_exclude: #this includes the dummy variables (like genres and original_language)
#         list_of_features.append(column)

columns_to_include = ['top_actor_popularity (out of 10)', 'budget_in_millions', 'release_month']
for column in columns_to_include:
    list_of_features.append(column)

X = np.array(cleaned_df[list_of_features])
y = cleaned_df['rating']

poly = PolynomialFeatures(degree=3) #idk what would be a good degree to use here
X_poly = poly.fit_transform(X)
X_poly = X_poly[:, 1:] #gets rid of the columns of 1s

Xtrain, Xtest, ytrain, ytest = train_test_split(X_poly, y, test_size=0.3) # 30% test, 70% train
m = line_of_best_fit(Xtrain, ytrain)
poly_model = linreg_predict(Xtest, ytest, m)

print("Line of best fit: ", m)
print("MSE of polynomial model: ", poly_model['mse'])
print("R2 of polynomial model: ", poly_model['r2'])


Line of best fit:  [ 7.72775223e+00  7.77447968e-02  9.82592529e-04  8.21234727e-02
 -1.83262024e-02 -1.66852118e-03 -1.42288623e-02  4.95778373e-05
 -3.10828114e-04 -8.54752902e-03  1.61972210e-03  4.62022122e-04
 -1.53480253e-04 -1.47285276e-05  7.49068037e-05  9.22887535e-04
 -4.41200663e-09  4.63015812e-07 -4.37240107e-06  2.79803623e-04]
MSE of polynomial model:  0.057594524359270045
R2 of polynomial model:  -0.08885987496595882


### Attempt 3 - Polynomial Regression with added interaction terms and dummy variables
#### Creating Fitting Model

$$
y = b_0 + b_1 x_1 + b_2 x_1^2 + b_3 x_1^3 + b_4 x_1^4 + b_5x_1x_3 + b_6x_2x_4 + b_7x_1x_4 + b_8x_2x_3
$$

Where:

- $y$: trip duration
- $x_1$: time of day
- $x_2$: member or casual (1 or 0)
- $x_3$: electric or classic bike (1 or 0)

- Polynomial terms $b_0 + b_1 x_1 + b_2 x_1^2 + b_3 x_1^3 + b_4 x_1^4$
- Interaction terms with dummy variables $b_5x_1x_3 + b_6x_2x_4 + b_7x_1x_4 + b_8x_2x_3$

## Part 2
(2%) A discussion of the preliminary results:
   - This may include checking of assumptions, generated plots/tables, measures of fit, or other attributes of the analysis
   - It does not have to be fully correct, but as a proof of concept must demonstrate that the group is close to completing the analysis

## THESE ARE THE EXAMPLES GIVEN!!

## Initial Approach
The initial question we proposed for Machine Learning part is "Can we predict the trip duration based on factors such as the time of day, user demographics, and starting/ending stations?", and after further discussions and analysis, we identified we can try building the models based on a Linear Regression for 3 features separately and a Polynomial Regression for one of the features. However, as we moved forward in with the Linear Regression analysis, we got unexpectedly large values for both $MSE$ and $R^2$ for all 3 features. This made us reconsider our initial attempt and go with Polynomial Regression for one of the features (Time of Day). After constructing the model equation and using cross-validation, we received values of $MSE$ = 627.869 and $R^2$ = 0.0035, which showed worse results, compared to Linear Regression. Therefore, we decided to fix our model by incorporating interactions with dummy variables (member_casual and rideable_type), which had values of 1 and 0. Building the polynomial regression model with interactions and dummy variables has slighly improved the $MSE$ = 599.699 and $R^2$ = 0.0482. However, the values were still significantly worse, than the Linear Regression model's ones. Therefore, we we will use the Linear Regression model to discuss the preliminary results.

## Model 1
First, we analyzed Model 1: Time of Day of Linear Regression model. We calculated $MSE$ = 514.094 and $R^2$ = 0.0477, where MSE value is lower than for the polynomial regression models and $R^2$ value is slightly lower than the second polynomial regression model. Based on the first plot of Residuals vs Fitted, we can see that the assumption of Linearity and Constant Variance is violated as the residuals do not follow a linear trend about 0 and also contain multiple outliers. The second plot of Residuals vs Order shows that there is some density of the data points near the y = 0 with mutliple outliers as well, indicating there might be a violation of Independence assumption as well. Looking at the Histogram of Residuals and Q-q plot, we can witness there is strong right-skewedness, showing that there is no normal distribution, meaning violation of Normality assumption. 

## Model 2
Second, we analyzed Model 2: User Demographics of Linear Regression Model. Calculating $MSE$ = 508.001 and $R^2$ = 0.059 shows that this is the highest values we were able to identify. There is a violation of Linearity and Constant Variance assumptions based on the Residuals vs Fitted graph, data points are not equally distributed around y=0 line, they have significant gaps between vertical line patterns of the data points. Moreover, the Independence assumption might be violated due to the density above y = 0 line. Finally, we see that the residuals are right skewed on Q-Q plot and Histogram of residuals. Therefore, the plots also violate the assumptions, despite better looking values.

## Model 3
Finally, the last Model 3: Starting and Ending Stations has $MSE$ = 514.076 and $R^2$ = 0.0478, the values are somewhat similar to the ones from Model 1. The model also fails to meet the assumptions of Linearity and Constant Variance assumptions based on the Residuals vs Fitted graph, data points are not distributed closer to y=0 line, have occasional gaps and outliers. Moreover, the Independence assumption might be violated due to the density above y = 0 line. Finally, we see that the residuals are right skewed on Q-Q plot and Histogram of residuals. 

## Conclusions
These findings paint a complex picture of the bike-sharing prediction challenge. While Linear Regression provided our best results among the attempted approaches, the consistent violations of key assumptions across all models reveal fundamental challenges in our modeling strategy. 

First, the pervasive violations of regression assumptions across all models suggest systemic issues in how we're approaching the prediction task. The violation of linearity assumptions, evidenced by clear patterns in our residual plots, indicates that the relationship between our predictors and trip duration isn't as straightforward as our models assume. For instance, the impact of time of day on trip duration might follow a more complex pattern influenced by rush hours, weekends, or seasonal effects that our linear approach can't capture. Similarly, the relationship between station locations and trip duration might be influenced by geographic features, traffic patterns, or neighborhood characteristics that require more sophisticated modeling approaches.

The heteroscedasticity observed in our models (non-constant variance in residuals) suggests that our predictions' accuracy varies significantly across different conditions. This could indicate that trip durations are more predictable in some circumstances than others – perhaps more consistent during regular commuting hours but more variable during leisure times. This varying predictability challenges our model's ability to provide reliable estimates across all scenarios.

The notably low R² values (ranging from 0.0477 to 0.059) are particularly telling. These values indicate that our models explain less than 6% of the variance in trip durations, leaving over 94% of the variation unexplained. This substantial unexplained variance could stem from several sources, such as missing key features, complex interaction, or hidden variables.

Future research directions might include:
- Exploring more sophisticated modeling techniques such as mixed-effects models to account for hierarchical structure in the data
- Investigating even more additional features that might better explain trip duration variability
- Considering non-parametric approaches that don't rely on the strict assumptions of linear regression
- Implementing geospatial modeling techniques to better capture the impact of station locations

 Understanding these limitations and potential improvements is crucial for developing more effective prediction models in future iterations. The complexity revealed by our analysis suggests that successful trip duration prediction might require a more nuanced, multi-model approach that can adapt to different conditions and user patterns. Our results demonstrate the challenges of real-world predictive modeling and the importance of thorough diagnostic testing, even when working with seemingly straightforward prediction tasks.